In [2]:
import os

data_loc = "../aclImdb"

def build_dataset(root):
    data = []
    labels = []
    for filename in os.listdir(os.path.join(root, 'pos')):
        data.append(open(os.path.join(root, 'pos', filename)).read())
        labels.append(1)
    for filename in os.listdir(os.path.join(root, 'neg')):
        data.append(open(os.path.join(root, 'neg', filename)).read())
        labels.append(0)
    return data, labels

class IMDBDataset(object):
    def __init__(self, root):
        self.train_data, self.train_labels = build_dataset(os.path.join(root, 'train'))
        self.dev_data, self.dev_labels = build_dataset(os.path.join(root, 'test'))

In [3]:
imdbDataset = IMDBDataset(data_loc)

In [4]:
from BagOfWordsNN import BoWMLP
clf = BoWMLP(imdbDataset.train_data, imdbDataset.train_labels, rare_word_threshold=15)
clf.fit()
clf.test(imdbDataset.dev_data, imdbDataset.dev_labels)

Done building vocab! Vocab size: 12314
Done building train dev!
Training...
Iteration 1, loss = 0.34067612
Iteration 2, loss = 0.15657234
Iteration 3, loss = 0.06402856
Iteration 4, loss = 0.01649610
Iteration 5, loss = 0.00646667
Iteration 6, loss = 0.00777718
Iteration 7, loss = 0.01170576
Iteration 8, loss = 0.01035176
Iteration 9, loss = 0.01007133
Iteration 10, loss = 0.00332316
Iteration 11, loss = 0.00200266
Iteration 12, loss = 0.00059611
Iteration 13, loss = 0.00044441
Iteration 14, loss = 0.00042735
Iteration 15, loss = 0.00042113
Iteration 16, loss = 0.00041667
Iteration 17, loss = 0.00041297
Iteration 18, loss = 0.00040972
Iteration 19, loss = 0.00040679
Iteration 20, loss = 0.00040407
Iteration 21, loss = 0.00040149
Iteration 22, loss = 0.00039900
Iteration 23, loss = 0.00039657
Iteration 24, loss = 0.00039418
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Training done!
Testing on dev...
Done! Scores below:
0.8732
0.85664


In [5]:
import joblib

joblib.dump(clf, "BoWMLP-IMDB.joblib")

['BoWMLP-IMDB.joblib']

In [ ]:
from BagOfWordsLogReg import BoWLR
clf2 = BoWLR(imdbDataset.train_data, imdbDataset.train_labels, rare_word_threshold=15)
clf2.fit()

In [8]:
clf2.test(imdbDataset.dev_data, imdbDataset.dev_labels)

0.86056


In [10]:
joblib.dump(clf2, "BoWLogReg-IMDB.joblib")

['BoWLogReg-IMDB.joblib']